[Link to tutorial](https://juliaai.github.io/DataScienceTutorials.jl/getting-started/ensembles-2/)

# Ensemble models (2)

In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/Documents/Personal Stuff/Repos/mike_scratch/mlj_tutorial/A-ensembles-2`


┌ Warning: The active manifest file has dependencies that were resolved with a different julia version (1.7.1). Unexpected behavior may occur.
└ @ nothing /Users/michaelherman/Documents/Personal Stuff/Repos/mike_scratch/mlj_tutorial/A-ensembles-2/Manifest.toml:0


Here we're going to build a random forest regressor on the Boston data. We've done random forest and ensemble modeling in previous tutorials, so this is largely practice. I'm going to code it with minimal commentary.

In [2]:
using MLJ
using PyPlot
using PrettyPrinting
using StableRNGs
import DataFrames: DataFrame, describe

In [3]:
X, y = @load_boston
sch = schema(X)
p = length(sch.names)
describe(y)

Summary Stats:
Length:         506
Missing Count:  0
Mean:           22.532806
Minimum:        5.000000
1st Quartile:   17.025000
Median:         21.200000
3rd Quartile:   25.000000
Maximum:        50.000000


Type:           Float64


In [4]:
DecisionTreeRegressor = @load DecisionTreeRegressor pkg=DecisionTree

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/michaelherman/.julia/packages/MLJModels/tMgLW/src/loading.jl:168


import MLJDecisionTreeInterface ✔


MLJDecisionTreeInterface.DecisionTreeRegressor

In [6]:
tree = machine(DecisionTreeRegressor(), X, y)
e = evaluate!(tree,  resampling=Holdout(fraction_train=0.8), measure=[rms, rmslp1])
e

PerformanceEvaluation object with these fields:
  measure, measurement, operation, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_pairs
Extract:
┌───────────────────────────────────────────────────┬─────────────┬─────────────
│ measure                                           │ measurement │ operation  ⋯
├───────────────────────────────────────────────────┼─────────────┼─────────────
│ RootMeanSquaredError()                            │ 7.06        │ predict    ⋯
│ RootMeanSquaredLogProportionalError(offset = 1.0) │ 0.328       │ predict    ⋯
└───────────────────────────────────────────────────┴─────────────┴─────────────
                                                                1 column omitted


Above is a single decision tree. For a random forest, we need a bagging ensemble.

In [8]:
forest = EnsembleModel(model=DecisionTreeRegressor(n_subfeatures=3))

DeterministicEnsembleModel(
    model = DecisionTreeRegressor(
            max_depth = -1,
            min_samples_leaf = 5,
            min_samples_split = 2,
            min_purity_increase = 0.0,
            n_subfeatures = 3,
            post_prune = false,
            merge_purity_threshold = 1.0,
            rng = Random._GLOBAL_RNG()),
    atomic_weights = Float64[],
    bagging_fraction = 0.8,
    rng = Random._GLOBAL_RNG(),
    n = 100,
    acceleration = CPU1{Nothing}(nothing),
    out_of_bag_measure = Any[])

In [9]:
rng = StableRNG(71)
m = machine(forest, X, y)
r = range(forest, :n, lower=10, upper=1000)
curves = learning_curve!(m, resampling=Holdout(fraction_train=0.8, rng=rng), range=r, measure=rms);

┌ Info: Training Machine{DeterministicTunedModel{Grid,…},…}.
└ @ MLJBase /Users/michaelherman/.julia/packages/MLJBase/MuLnJ/src/machines.jl:464
┌ Info: Attempting to evaluate 30 models.
└ @ MLJTuning /Users/michaelherman/.julia/packages/MLJTuning/Al9yX/src/tuned_models.jl:680


Evaluating over 30 metamodels:   0%[>                        ]  ETA: N/A

Evaluating over 30 metamodels:   3%[>                        ]  ETA: 0:01:55

Evaluating over 30 metamodels:   7%[=>                       ]  ETA: 0:01:07

Evaluating over 30 metamodels:  10%[==>                      ]  ETA: 0:00:44

...

Evaluating over 30 metamodels:  87%[=====================>   ]  ETA: 0:00:04

Evaluating over 30 metamodels:  90%[======================>  ]  ETA: 0:00:03

Evaluating over 30 metamodels:  93%[=======================> ]  ETA: 0:00:02

Evaluating over 30 metamodels:  97%[========================>]  ETA: 0:00:01

Evaluating over 30 metamodels: 100%[=========================] Time: 0:00:31


Plotting the learning curve will help us determine the ideal number of trees.

But since plotting is still busted in this env, we'll just find the record with the minimum measurement.

In [38]:
hcat(curves.parameter_values, curves.measurements)[findmin(curves.measurements)[2],:]

2-element Vector{Float64}:
 727.0
   3.7328328312671166

We'll use 727 trees.

In [39]:
forest.n = 727

727

In [40]:
params(forest) |> pprint

(model = (max_depth = -1,
          min_samples_leaf = 5,
          min_samples_split = 2,
          min_purity_increase = 0.0,
          n_subfeatures = 3,
          post_prune = false,
          merge_purity_threshold = 1.0,
          rng = Random._GLOBAL_RNG()),
 atomic_weights = [],
 bagging_fraction = 0.8,
 rng = Random._GLOBAL_RNG(),
 n = 727,
 acceleration = CPU1{Nothing}(nothing),
 out_of_bag_measure = [])

In [41]:
r_sf = range(forest, :(model.n_subfeatures), lower=1, upper=12)
r_bf = range(forest, :bagging_fraction, lower=0.4, upper=1.0);

tuned_forest = TunedModel(model=forest,
                          tuning=Grid(resolution=3),
                          resampling=CV(nfolds=6, rng=StableRNG(71)),
                          ranges=[r_sf,r_bf],
                          measures=rms)
m = machine(tuned_forest, X, y)
e = evaluate!(m, resampling=Holdout(fraction_train=0.8),
              measure=[rms, rmslp1])
e

PerformanceEvaluation object with these fields:
  measure, measurement, operation, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_pairs
Extract:
┌───────────────────────────────────────────────────┬─────────────┬─────────────
│ measure                                           │ measurement │ operation  ⋯
├───────────────────────────────────────────────────┼─────────────┼─────────────
│ RootMeanSquaredError()                            │ 3.99        │ predict    ⋯
│ RootMeanSquaredLogProportionalError(offset = 1.0) │ 0.252       │ predict    ⋯
└───────────────────────────────────────────────────┴─────────────┴─────────────
                                                                1 column omitted


In [62]:
r = report(m)
res = r.plotting
hcat(res.parameter_values[:,1:2], res.measurements)

9×3 Matrix{Any}:
 12  0.7  4.10075
 12  1.0  4.46944
  1  1.0  4.71862
  6  1.0  3.83574
  6  0.4  4.18354
 12  0.4  4.14608
  1  0.7  4.98
  6  0.7  3.92241
  1  0.4  5.45518

The minimum occurred at 6 subfeatures and a bagging fraction of 1.0.

Now we predict.

In [64]:
ŷ = predict(m, X)    # y\hat + tab => ŷ
@show rms(ŷ, y);

rms(ŷ, y) = 2.407968274450449
